In [1]:
!pip install pygame
import pygame
import random
import sys

pygame.init()


pygame 2.5.2 (SDL 2.28.3, Python 3.9.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


(5, 0)

In [2]:
class Color:
  RED = (255,0,0)
  BLUE = (0,0,255)
  YELLOW = (255,255,0)
  BLACK = (0,0,0)


In [3]:
class Player:
  def __init__(self, x, y, size, color=Color.RED):
    self.x = x
    self.y = y
    self.size = size
    self.color = color

  def draw(self, screen):
    pygame.draw.rect(screen, self.color, (self.x, self.y, self.size, self.size))


  def detect_collision(self, other):

    if (other.x >= self.x and other.x < (self.x + self.size)) or (self.x >= other.x and self.x < (other.x + other.size)):
      if (other.y >= self.y and other.y < (self.y + self.size)) or (self.y >= other.y and self.y < (other.y + other.size)):
        return True
    return False

In [4]:
class Enemy(Player): #It will take from the player class
  def __init__(self, x, y):
    super().__init__(x, y, size=50, color=Color.BLUE)

In [5]:
class LargeEnemy(Player): #It will take from the player class
  def __init__(self, x, y):
    super().__init__(x, y, size=100, color=Color.BLUE)

In [6]:
class Human(Player): #It will take from the player class
  def __init__(self, x, y):
    super().__init__(x, y, size=50, color=Color.RED)

In [7]:
class Screen:
  def __init__(self, width=800, height=600, background_Color = Color.BLACK, font_type='monospace', font_size=35, clock_tick=30):
    self.width = width
    self.height = height
    self.background_Color = background_Color
    self.screen = pygame.display.set_mode((width, height))
    self.font = pygame.font.SysFont(font_type, font_size)
    self.clock = pygame.time.Clock()
    self.clock_tick = clock_tick


  def refresh_background(self):
    self.screen.fill(self.background_Color)

  def draw_enemies(self, enemy_list):
    for enemy in enemy_list:
      enemy.draw(self.screen)

  def draw_player(self, player):
    player.draw(self.screen)

  def draw_score_label(self, score, labelcolor=Color.YELLOW):
    text = f"Score: {score}"
    label = self.font.render(text, 1, labelcolor)
    self.screen.blit(label, (self.width-200, self.height-40))

  def update_screen(self, enemy_list, player, score): #A connected function to call everything together
    self.refresh_background()
    self.draw_enemies(enemy_list)
    self.draw_player(player)
    self.draw_score_label(score)
    self.clock.tick(self.clock_tick)
    pygame.display.update()

In [8]:
class Game:
  def __init__(self, speed=10, score=0, max_enemies=10, delay=0.1):
    self.speed = speed
    self.score = score
    self.max_enemies = max_enemies
    self.delay = delay
    self.enemy_list = []

  def drop_enemies(self, screen_width):
    delay = random.random()
    if len(self.enemy_list) < self.max_enemies and delay < self.delay:
      randomx = random.randint(0, screen_width)
      y_pos = 0
      enemy = Enemy(randomx, y_pos)
      self.enemy_list.append(enemy) #Getter/Setter wise, the enemy_list is directly set/updated

  def update_enemy_positions(self, screen_height):
    new_enemy_list = []
    for enemy in self.enemy_list:
      if enemy.y >= 0 and enemy.y < screen_height:
        enemy.y += self.speed
        new_enemy_list.append(enemy)
      else:
        self.score += 1           #Getter/Setter wise, the score is directly set/updated
    self.enemy_list = new_enemy_list

  def set_level(self):
    if self.score < 20:
      self.speed = 5
    elif self.score < 40:
      self.speed = 8
    elif self.score < 60:
      self.speed = 12
    else:
      self.speed = 15

  def collision_check(self, player):
    for enemy in self.enemy_list:
      if enemy.detect_collision(player):
        return True
    return False

In [9]:
game_over = False

screen = Screen()
player = Human(screen.width/2, screen.height-100)
game = Game()

while not game_over:

  for event in pygame.event.get():
    if event.type == pygame.QUIT:
      sys.exit()

    if event.type == pygame.KEYDOWN:

      if event.key == pygame.K_LEFT:
        player.x -= player.size            #Directly setting the position
      elif event.key == pygame.K_RIGHT:
        player.x += player.size            #Directly setting the position

  game.drop_enemies(screen.width)             #This line create item in the enemy_list
  game.update_enemy_positions(screen.height)  #This line update item in the enemy_list, score is updated as well
  game.set_level()                           #Hence this line is able to access the updated score abstracted in the Game class.
  screen.update_screen(game.enemy_list, player, game.score) #And Hence this line is able to access the updated enemy_list abstracted in the Game class, which is game.enemy_list


  if game.collision_check(player):
    game_over = True
    break